In [ ]:

import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from NZB.search import NZBSearch
from NZB.sabnzbd import SABnzbdClient

searcher = NZBSearch()

def format_str_iso_date_as_yy_mm_dd(iso_date_str):
    from datetime import datetime
    parsed_date = datetime.strptime(iso_date_str, "%Y-%m-%d")
    return format_iso_date_as_yy_mm_dd(parsed_date)

def format_iso_date_as_yy_mm_dd(iso_date):
    return iso_date.strftime("%y %m %d")    

def format_studio_name(studio_name):
    return studio_name.replace(" ", "").replace("-", "")

stash = get_stashapp_client("MISSING_SDB_")
stash_client = StashAppClient("MISSING_SDB_")

In [2]:
# StashDB
from libraries.StashDbClient import StashDbClient
import dotenv
import os

dotenv.load_dotenv()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

In [ ]:
performer_id = stash.find_performer("Alexis Crystal")['id']

target_scenes: pl.DataFrame = pl.DataFrame(stash.find_scenes({ "performers": { "value": [performer_id], "modifier": "INCLUDES" }}, fragment="id title date details studio { id name } performers { id name }"))
target_scenes = target_scenes.sort(by=["studio", "date"])
target_scenes = target_scenes.filter(pl.col("studio").struct.field("name").str.contains("Fitting-Room"))
target_scenes



In [84]:
stashapp_scene_id = target_scenes.to_dicts()[1]['id']

In [ ]:
stashapp_target_scene = stash.find_scene(stashapp_scene_id, fragment="id title details urls date performers { id name stash_ids { stash_id endpoint } } studio { id name } stash_ids { stash_id endpoint }")
print(f"{stashapp_target_scene['id']} {stashapp_target_scene['title']} {[performer['name'] for performer in stashapp_target_scene['performers']]}")

stashdb_ids = [stash_id['stash_id'] for stash_id in stashapp_target_scene['stash_ids'] if stash_id['endpoint'] == "https://stashdb.org/graphql"]
stashdb_id = stashdb_ids[0] if stashdb_ids else None

stashdb_scenes = stashbox_client.query_scenes([stashdb_id])
stashdb_scenes

stashdb_target_scene = stashdb_scenes.to_dicts()[0]
stashdb_target_scene

print(f"{stashdb_target_scene['id']} {stashdb_target_scene['title']} {[performer['as'] or performer['performer']['name'] for performer in stashdb_target_scene['performers']]}")

# Get primary performer IDs from StashApp scene
primary_performer_ids = [
    stash_id['stash_id'] 
    for performer in stashapp_target_scene['performers'] 
    for stash_id in performer['stash_ids']
    if stash_id['endpoint'] == "https://stashdb.org/graphql"
]
print(f"Primary performer IDs: {primary_performer_ids}")

# Get matching performers from StashDB scene
stashdb_primary_performers = [
    performer 
    for performer in stashdb_target_scene['performers'] 
    if performer['performer']['id'] in primary_performer_ids
]
print(f"\nMatched performers: {[p['performer']['name'] for p in stashdb_primary_performers]}")

stashdb_primary_performer_names = [performer['as'] or performer['performer']['name'] for performer in stashdb_primary_performers]
stashdb_primary_performer_names

search_query = f"{format_studio_name(stashdb_target_scene['studio']['name'])} \"{format_iso_date_as_yy_mm_dd(stashdb_target_scene['date'])}\""
search_query


print(stashdb_target_scene)
print(search_query)

performer_search_query = f"{stashdb_primary_performer_names[0]} \"{format_iso_date_as_yy_mm_dd(stashdb_target_scene['date'])}\""
performer_search_query

print(performer_search_query)


In [61]:
search_query = "hersexdebut sata"

In [ ]:
results = searcher.search(search_query)
if results.is_empty():
    results = searcher.search(performer_search_query)
    
results


In [87]:
first_result = results.to_dicts()[0]

In [88]:
sab = SABnzbdClient()
# Add the download
result = sab.add_nzb_url(first_result['link'], first_result['title'])

# if result['status']:
#     if 'nzo_id' in result:
#         # Wait for download to complete
#         download_result = sab.wait_for_completion(result['nzo_id'])
#         if download_result['status'] == 'completed':
#             print(f"Download completed! File saved to: {download_result['path']}")
#         else:
#             print(f"Download failed: {download_result.get('error', 'Unknown error')}")
#     else:
#         print(f"Added to SABnzbd but couldn't get job ID: {result.get('error')}")
# else:
#     print(f"Failed to add to SABnzbd: {result.get('error')}")

In [ ]:
import os

# Get all files in download directory
files = os.listdir(download_result['path'])

# Get full paths and sizes
file_info = []
for f in files:
    full_path = os.path.join(download_result['path'], f)
    if os.path.isfile(full_path):
        size = os.path.getsize(full_path)
        file_info.append((full_path, size))

# Sort by size descending
file_info.sort(key=lambda x: x[1], reverse=True)

# Get largest video file
video_extensions = {'.mp4', '.mkv', '.avi', '.wmv', '.mov'}
for filepath, size in file_info:
    ext = os.path.splitext(filepath)[1].lower()
    if ext in video_extensions:
        converted_filepath = filepath
        break
else:
    raise Exception("No video file found in download directory")

print(f"Using video file: {converted_filepath} ({size/1024/1024:.1f} MB)")


In [ ]:
import subprocess
import json

process = subprocess.run(
    ['C:\\Tools\\videohashes-windows-amd64.exe', '-json', converted_filepath,], 
    capture_output=True,  # Captures both stdout and stderr
    text=True  # Returns strings instead of bytes
)
assert process.returncode == 0, f"Failed to run videohashes: {process.stderr}"
videohashes_data = json.loads(process.stdout)
videohashes_data

In [ ]:
videohashes_data["phash"] in [fingerprint["hash"] for fingerprint in target_scene['fingerprints'] if fingerprint["algorithm"] == "PHASH"]